# **Harris corner detector**
Obiettivo dell’esercitazione è la realizzazione di un algoritmo per la localizzazione dei corner basato **metodo di Harris**.

In particolare, data in input un’immagine RGB, l’algoritmo dovrà restituire un array di punti corrispondenti ai principali corner rilevati nell’immagine.

Il metodo di Harris si basa interamente sull’analisi del gradiente, quindi l’analisi viene fatta sulla versione grayscale dell’immagine in input.

# **Import delle librerie**
È necessario eseguire l'import delle librerie utilizzate durante l'esercitazione.

In [1]:
import cv2
import numpy as np

# **Caricamento immagine**
Il metodo di Harris si basa interamente sull’**analisi del gradiente**, quindi l’analisi viene fatta sulla versione **grayscale** dell’immagine in input.

# **Harris corner detector in OpenCV**
La libreria OpenCV mette a disposizione la funzione `cv2.cornerHarris()` per la localizzazione dei corner (documentazione [qui](https://docs.opencv.org/4.x/dc/d0d/tutorial_py_features_harris.html)). I parametri sono:

* `img`: immagine di input grayscale (float32 type).
* `blockSize`: dimensione dell'intorno da considerare per la localizzazione dei corner
* `ksize`: dimensione del filtro per il calcolo del gradiente con Sobel
* `k`: parametro $\alpha$ nell'equazione dell'Harris corner detector.

In [3]:
img = cv2.imread('./output/370046.png')
cv2.imshow("Result Image", img)
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
dst = cv2.cornerHarris(img_gray,2,3,0.04)
#dst = cv2.dilate(dst, None)
img[dst>0.01*dst.max()]=[0,0,255]

: 

## Corner with SubPixel Accuracy
A volte potrebbe essere necessario individuare i corner con un'accuratezza maggiore. La libreria OpenCV mette a disposizione la funzione `cv2.cornerSubPix()` che raffina ulteriormente i corner localizzati con l'algoritmo di base.

È necessario innanzitutto individuare i corner con il metodo `cv2.cornerHarris()` e poi **clusterizzarli** per utilizzare i centroidi dei cluster individuati come corner finali (potrebbe esserci un gruppo di punti che fanno riferimento allo stesso corner).

È necessario fissare un **criterio d'arresto** per l'algoritmo di clustering (un numero prefissato di iterazioni o un certo valore di accuratezza). L'algoritmo di clustering richiede anche di specificare la dimensione dell'intorno da considerare per la ricerca dei corner.

In [ ]:
img = cv2.imread('scacchiera.png')
cv2_imshow(img)
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
dst = cv2.cornerHarris(img_gray,2,3,0.04)
ret, dst = cv2.threshold(dst,0.01*dst.max(),255,0)
dst = np.uint8(dst)

cv2_imshow(dst)

# find centroids
ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)

# define the criteria to stop and refine the corners
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
corners = cv2.cornerSubPix(img_gray,np.float32(centroids),(5,5),(-1,-1),criteria)

# Now draw them
for c in corners:
    cv2.circle(img, (int(c[0]), int(c[1])), radius=3, color=(0, 255, 0), thickness=1)
cv2_imshow(img)

# **La nostra implementazione**
## Caricamento dell'immagine

In [ ]:
img = cv2.imread('scacchiera.png')
#img = cv2.imread('scacchiera.png', cv2.IMREAD_GRAYSCALE) lettura grayscale

cv2_imshow(img)
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

## Calcolo del gradiente e blurring
La ricerca dei corner si basa sull’analisi del coefficiente di cornerness. Per un generico punto $(𝑥,𝑦)$ il valore si ottiene come:

$R=det(M)-\alpha\cdot tr(M)^2$

dove:
*   $\alpha$ è una costante che normalmente assume valori nel range $[0.04, 0.06]$
*   $M$ è una matrice $2 \times 2$ calcolata a partire dal gradiente dell'immagine come segue:

$M=\sum_{x,y} w(x,y) \begin{bmatrix} \nabla_x^2(x,y) &\nabla_x(x,y) \nabla_y(x,y)\\\nabla_x(x,y) \nabla_y(x,y)&\nabla_y^2(x,y) \end{bmatrix}$

Il primi step dell'algoritmo sono dunque:
* calcolo del gradiente dell'immagine in $x$ ($\nabla_x$) e in $y$ ($\nabla_y$). Anche in questo caso possiamo usare Sobel.
* calcolo di $\nabla_x^2$, $\nabla_y^2$, $\nabla_x\nabla_y$

NOTE:
* per visualizzare le immagini è necessario normalizzarle nel giusto range di valori usando la funzione `cv2.normalize`, con metodo di normalizzazione `cv2.NORM_MINMAX`.

<img src=https://biolab.csr.unibo.it/vr/esercitazioni/NotebookImages/EsHarris/step1.png width="800">

In [4]:
img_gray = cv2.imread("./output/370046.png", cv2.IMREAD_GRAYSCALE)
gx = cv2.Sobel(img_gray,cv2.CV_64F,1,0,ksize=3)
gy = cv2.Sobel(img_gray,cv2.CV_64F,0,1,ksize=3)
gx2 = np.multiply(gx, gx)
gy2 = np.multiply(gy, gy)
gxgy = np.multiply(gx, gy)

if gxgy is None:
    print("Errore nel caricamento dell'immagine.")
else:
    # Mostra l'immagine in scala di grigi
    cv2.imshow('Immagine Grayscale', gx2)

    # Attende fino a quando l'utente non preme un tasto
    cv2.waitKey(0)

    # Chiude tutte le finestre di OpenCV
    cv2.destroyAllWindows()

Come visto in teoria, l'applicazione della funzione finestra può essere realizzata nella pratica con un'operazione di smoothing sulle immagini $\nabla_x^2$, $\nabla_y^2$ e $\nabla_x\nabla_y$ calcolate precedentemente.

Per lo smoothing Gaussiano utilizzare la funzione `cv2.GaussianBlur` con i parametri già suggeriti nel codice.

<img src=https://biolab.csr.unibo.it/vr/esercitazioni/NotebookImages/EsHarris/step2.png width="800">

In [7]:
filter_size = 7
filter_sigma = 1.4

gx2_b = cv2.GaussianBlur(gx2,(filter_size,filter_size),filter_sigma, cv2.BORDER_DEFAULT)
gy2_b = cv2.GaussianBlur(gy2,(filter_size,filter_size),filter_sigma, cv2.BORDER_DEFAULT)
gxgy_b = cv2.GaussianBlur(gxgy,(filter_size,filter_size),filter_sigma, cv2.BORDER_DEFAULT)
#cv2_imshow(cv2.normalize(gx2_b, None, 0, 255, cv2.NORM_MINMAX))
#cv2_imshow(cv2.normalize(gy2_b, None, 0, 255, cv2.NORM_MINMAX))
#cv2_imshow(cv2.normalize(gxgy_b, None, 0, 255, cv2.NORM_MINMAX))

if gxgy is None:
    print("Errore nel caricamento dell'immagine.")
else:
    # Mostra l'immagine in scala di grigi
    cv2.imshow('Immagine Grayscale', cv2.normalize(gx2_b, None, 0, 255, cv2.NORM_MINMAX))

    # Attende fino a quando l'utente non preme un tasto
    cv2.waitKey(0)

    # Chiude tutte le finestre di OpenCV
    cv2.destroyAllWindows()

## Calcolo cornerness map
A partire dalla versione blurred di $\nabla_x^2$, $\nabla_y^2$ e $\nabla_x\nabla_y$ è ora possibile calcolare la cornerness map.

Si consiglia di inizializzare la cornerness map a 0 e di *memorizzare solo i valori di cornerness superiori alla soglia* preimpostata.

<img src=https://biolab.csr.unibo.it/vr/esercitazioni/NotebookImages/EsHarris/step3.png>

In [8]:
cornerness_thr = 1000000
alfa = 0.04

def compute_cornerness_map(gx2_b, gy2_b, gxgy_b):
  map = np.zeros(gx2_b.shape)
  for p, x in np.ndenumerate(map):
    M = np.zeros((2,2))
    M[0,0] = gx2_b[p]
    M[0,1] = M[1,0] = gxgy_b[p]
    M[1,1] = gy2_b[p]
    det = np.linalg.det(M)
    tr = np.trace(M)
    r = det - alfa * tr * tr
    if r > cornerness_thr:
      map[p] = r
  return map

c_map = compute_cornerness_map(gx2_b, gy2_b, gxgy_b)
#cv2_imshow(cv2.normalize(c_map, None, 0, 255, norm_type=cv2.NORM_MINMAX))

if c_map is None:
    print("Errore nel caricamento dell'immagine.")
else:
    # Mostra l'immagine in scala di grigi
    cv2.imshow('Immagine Grayscale', cv2.normalize(c_map, None, 0, 255, norm_type=cv2.NORM_MINMAX))

    # Attende fino a quando l'utente non preme un tasto
    cv2.waitKey(0)

    # Chiude tutte le finestre di OpenCV
    cv2.destroyAllWindows()

In [9]:
cornerness_thr = 0.01
alfa = 0.04

def compute_cornerness_map(gx2_b, gy2_b, gxgy_b):
  map = np.zeros(gx2_b.shape)
  for p, x in np.ndenumerate(map):
    M = np.zeros((2,2))
    M[0,0] = gx2_b[p]
    M[0,1] = M[1,0] = gxgy_b[p]
    M[1,1] = gy2_b[p]
    det = np.linalg.det(M)
    tr = np.trace(M)
    r = det - alfa * tr * tr
    map[p] = r
  map[map<cornerness_thr*map.max()]=0
  return map

c_map = compute_cornerness_map(gx2_b, gy2_b, gxgy_b)
#cv2_imshow(cv2.normalize(c_map, None, 0, 255, norm_type=cv2.NORM_MINMAX))
if c_map is None:
    print("Errore nel caricamento dell'immagine.")
else:
    # Mostra l'immagine in scala di grigi
    cv2.imshow('Immagine Grayscale', cv2.normalize(c_map, None, 0, 255, norm_type=cv2.NORM_MINMAX))

    # Attende fino a quando l'utente non preme un tasto
    cv2.waitKey(0)

    # Chiude tutte le finestre di OpenCV
    cv2.destroyAllWindows()

## Soppressione dei non massimi

L'idea alla base dell'algoritmo è far scorrere una finestra di dimensione prefissata su tutta la mappa. Se esiste, all'interno della finestra, un pixel con un valore di cornerness superiore al pixel centrale, allora il pixel centrale viene "soppresso".

Alla fine sopravviveranno alla scrematura solo i pixel che rappresentano dei massimi locali.

<img src=https://biolab.csr.unibo.it/vr/esercitazioni/NotebookImages/EsHarris/step4.png>

In [11]:
window_radius = 5

def non_maxima_suppression(map):
  s = map.shape
  max_y = s[0] - window_radius
  max_x = s[1] - window_radius
  points = []
  for y in range(window_radius, max_y):
    for x in range(window_radius, max_x):
      current_value = map[y, x]
      if (current_value > 0):
        found = False
        for i in range(-window_radius, window_radius):
          for j in range(-window_radius, window_radius):
            if map[y+i, x+j] > current_value:
             found = True
             break
        if not found:
          points.append((x, y))
  return points

def draw_corners(img, corners):
  res = img.copy()
  for c in corners:
    cv2.circle(res, c, radius=3, color=(0, 0, 255), thickness=1)
  return res

corners = non_maxima_suppression(c_map)
res = draw_corners(img_gray, corners)
#cv2_imshow(res)
if res is None:
    print("Errore nel caricamento dell'immagine.")
else:
    # Mostra l'immagine in scala di grigi
    cv2.imshow('Immagine Grayscale', res)

    # Attende fino a quando l'utente non preme un tasto
    cv2.waitKey(0)

    # Chiude tutte le finestre di OpenCV
    cv2.destroyAllWindows()